In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


2021-11-01 22:45:32.265150: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-01 22:45:32.265170: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
TRAIN_DATA_PATH = "data/Q1_train.csv"
TEST_DATA_PATH = "data/Q1_test.csv"

SIZE_IN_ROW = 28
VALIDATION_PERCNTAGE = 0.2

LABELS = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',10:'K',11:'L',
          12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',
          19:'T',20:'U',21:'V',22:'W',23:'X',24:'Y'}


WIDTH = 28
HEIGHT = 28

In [3]:
all_data = pd.read_csv(TRAIN_DATA_PATH,index_col=False)
test_data = pd.read_csv(TEST_DATA_PATH,index_col=False)

In [4]:
all_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [5]:
test_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,6,149,149,150,150,150,151,151,150,151,...,138,148,127,89,82,96,106,112,120,107
1,5,126,128,131,132,133,134,135,135,136,...,47,104,194,183,186,184,184,184,182,180
2,10,85,88,92,96,105,123,135,143,147,...,68,166,242,227,230,227,226,225,224,222
3,0,203,205,207,206,207,209,210,209,210,...,154,248,247,248,253,236,230,240,253,255
4,3,188,191,193,195,199,201,202,203,203,...,26,40,64,48,29,46,49,46,46,53


In [6]:
# for data in [(index,data) for index, data in all_data.iterrows() if index in range(25)]:
#   plt.imshow(np.array(data[1][1:]).reshape(WIDTH,HEIGHT),cmap='gray')
#   plt.title(LABELS[data[1][0]])
#   plt.show()


In [7]:
## random shuffling first time
all_data = all_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

In [8]:
data_length = all_data.shape[0]
test_data_length = test_data.shape[0]
print(data_length)
validation_data_length = int(data_length * VALIDATION_PERCNTAGE)
train_data_length = data_length - validation_data_length

print(train_data_length,validation_data_length)

27455
21964 5491


In [9]:
y_labels = all_data['label']
all_data = all_data.drop('label',1)

y_test_labels = test_data['label']
test_data = test_data.drop('label',1)


/tmp/ipykernel_40972/1524214916.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  all_data = all_data.drop('label',1)
/tmp/ipykernel_40972/1524214916.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  test_data = test_data.drop('label',1)


In [10]:
## DATA normalization

x_train = all_data.copy().values[:train_data_length].astype(np.float32)
_y_train = y_labels.copy()[:train_data_length]
x_std = np.std(x_train)
x_mean = np.mean(x_train)

print(x_train.shape)

for i in range(x_train.shape[0]):
    x_train[i]  = (x_train[i] - x_mean)/x_std

print(x_train)
print(x_mean,x_std)

x_valid = all_data.copy().values[train_data_length:].astype(np.float32)
_y_valid = y_labels.copy().values[train_data_length:]

for i in range(x_valid.shape[0]):
    x_valid[i]  = (x_valid[i] - x_mean)/x_std


y_train =  tf.Session().run(tf.one_hot(_y_train,25))
y_valid =  tf.Session().run(tf.one_hot(_y_valid,25))

x_test = test_data.copy().values.astype(np.float32)
y_test = tf.Session().run(tf.one_hot(y_test_labels.values,25))
print('x test:',x_test.shape)
for i in range(x_test.shape[0]):
    x_test[i]  = (x_test[i] - x_mean)/x_std

print(x_train)
print(x_valid)
print(x_test)

(21964, 784)
[[-0.0651551   0.01682283  0.07830627 ...  1.0210525   1.000558
   1.0210525 ]
 [ 0.81610763  0.81610763  0.81610763 ...  1.3079752   1.2874807
   1.2874807 ]
 [-0.35207784 -0.37257233 -0.37257233 ... -3.2622943  -3.2622943
  -3.2622943 ]
 ...
 [ 0.69314075  0.7341297   0.7136352  ...  1.2259973   1.3284696
   1.4514366 ]
 [-1.0693847  -0.9669123  -0.8234509  ...  0.57017386  0.5906683
   0.5906683 ]
 [ 0.22176765  0.24226213  0.34473455 ...  1.1645138   1.1440194
   1.1235249 ]]
159.17915 48.79362
x test: (7172, 784)
[[-0.0651551   0.01682283  0.07830627 ...  1.0210525   1.000558
   1.0210525 ]
 [ 0.81610763  0.81610763  0.81610763 ...  1.3079752   1.2874807
   1.2874807 ]
 [-0.35207784 -0.37257233 -0.37257233 ... -3.2622943  -3.2622943
  -3.2622943 ]
 ...
 [ 0.69314075  0.7341297   0.7136352  ...  1.2259973   1.3284696
   1.4514366 ]
 [-1.0693847  -0.9669123  -0.8234509  ...  0.57017386  0.5906683
   0.5906683 ]
 [ 0.22176765  0.24226213  0.34473455 ...  1.1645138   1.14

2021-11-01 22:45:39.295420: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-01 22:45:39.295475: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mohy): /proc/driver/nvidia/version does not exist
2021-11-01 22:45:39.296045: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
## HYPER PARAMS
learning_rate = 0.001
batch_size = 1000
n_inputs = x_train.shape[1]
layers_num = [256,256,256] ## hidden layers
n_outputs = 25

weights = []
biases = []
layers = []


layers_num.insert(0,n_inputs)
layers_num.append(n_outputs)

tf.reset_default_graph()

# tf.summary.FileWriter('./graphs', tf.get_default_graph())


X = tf.placeholder('float', [None, n_inputs])
Y = tf.placeholder('float', [None, n_outputs])

histogram_tensorboard = []

print(layers_num)
for i in range(len(layers_num)-1):
    weights.append(tf.get_variable(dtype=tf.float32, shape=(layers_num[i],layers_num[i+1]),name="W_{}".format(i),
                                   initializer=tf.truncated_normal_initializer(stddev=0.001)))
    histogram_tensorboard.append(tf.summary.histogram("W_{}".format(i), weights[i]))
    
    biases.append(tf.get_variable(dtype=tf.float32, shape=(layers_num[i+1]) , name="B_{}".format(i),
                                  initializer=tf.zeros_initializer()))
    histogram_tensorboard.append(tf.summary.histogram("B_{}".format(i), biases[i]))
    
# histogram_merged = tf.summary.merge(histogram_tensorboard)
    
print(weights)
print(biases)

layers.append(tf.nn.relu(tf.add(tf.matmul(X, weights[0]), biases[0])))

layers.append(tf.nn.relu(tf.add(tf.matmul(layers[0], weights[1]), biases[1])))
layers.append(tf.nn.relu(tf.add(tf.matmul(layers[1], weights[2]), biases[2])))

layers.append(tf.matmul(layers[2], weights[3]) + biases[3])
# layers.append(tf.matmul(layers[2], weights[3]))

print(layers)

[784, 256, 256, 256, 25]
[<tf.Variable 'W_0:0' shape=(784, 256) dtype=float32_ref>, <tf.Variable 'W_1:0' shape=(256, 256) dtype=float32_ref>, <tf.Variable 'W_2:0' shape=(256, 256) dtype=float32_ref>, <tf.Variable 'W_3:0' shape=(256, 25) dtype=float32_ref>]
[<tf.Variable 'B_0:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'B_1:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'B_2:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'B_3:0' shape=(25,) dtype=float32_ref>]
[<tf.Tensor 'Relu:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'Relu_1:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'Relu_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'add_3:0' shape=(?, 25) dtype=float32>]


In [12]:
loss= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits= layers[-1],
                                            name='LOSS'))
## SGD
# optimizer = tf.train.GradientDescentOptimizer(learning_rate)
# training_op = optimizer.minimize(loss)

## ADAM

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(loss)


correct_pred = tf.equal(tf.argmax(layers[-1], 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


loss_tensorboard=tf.summary.scalar('Validation_Loss', loss)
acc_tensorboard=tf.summary.scalar('Accuracy_Loss', accuracy)
train_tensorboard=tf.summary.scalar('Training_Loss', loss)

merged = tf.summary.merge([loss_tensorboard,acc_tensorboard] + histogram_tensorboard)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [13]:
def train_task(summary_path,train_data,validate_data,batch_size,loss,accuracy,
               training_op,epoches,train_tensorboard,merged_tensorboard):
    loss_buffer = []
    acc_buffer = []
    with tf.Session() as sess:
        write = tf.summary.FileWriter(summary_path, sess.graph)

        sess.run(tf.global_variables_initializer())
        for epoch in range(0,epoches):
        
            for i in range(int(train_data_length/batch_size)):
                f={X:train_data[0][i*batch_size : (i+1)*batch_size] , 
                   Y:train_data[1][i*batch_size : (i+1)*batch_size] }
                _, train_val = sess.run([training_op, train_tensorboard],feed_dict=f)
                write.add_summary(train_val,epoch * int(train_data_length/batch_size) + i)

              # print('iter {}: \t Loss={:.2f}'.format(i,sess.run(loss,feed_dict=f)))

            f_valid={X:validate_data[0], Y:validate_data[1]}

            val,ls,acc = sess.run([merged_tensorboard,loss,accuracy], feed_dict=f_valid)
            write.add_summary(val,epoch)
            # write.add_summary(acc,epoch)
            if epoch%25 == 0:
                print('--------------------------------------------------')
                print('Epoch:{} \t Validation Loss:{:.2f} , acc: {:.2f}'.format(epoch,ls,acc))
                print('--------------------------------------------------')
            loss_buffer.append(ls)
            acc_buffer.append(acc)

        ls,acc = sess.run([loss,accuracy],feed_dict={X:x_test,Y:y_test})
        print('--------------------------------------------------')
        print('final \t test Loss:{:.2f} , acc: {:.2f}'.format(ls,acc))
        print('--------------------------------------------------')
        return (loss_buffer,acc_buffer)



In [14]:
# loss_buffer = []
# acc_buffer = []
# with tf.Session() as sess:
#     # sess = tf.InteractiveSession()
#     write = tf.summary.FileWriter('./summary/adam', sess.graph)

#     sess.run(tf.global_variables_initializer())
#     for epoch in range(0,1000):
#         # print('==================================================')
#         # print('====== Epoch=%s =================================== ' %(epoch))
#         # print('==================================================')
#         for i in range(int(train_data_length/batch_size)):
#             f={X:x_train[i*batch_size : (i+1)*batch_size] , 
#                Y:y_train[i*batch_size : (i+1)*batch_size] }
#             _, train_val = sess.run([training_op, train_tensorboard],feed_dict=f)
#             write.add_summary(train_val,epoch * int(train_data_length/batch_size) + i)
            
#           # print('iter {}: \t Loss={:.2f}'.format(i,sess.run(loss,feed_dict=f)))

#         f_valid={X:x_valid, Y:y_valid}
        
#         val,ls,acc = sess.run([merged,loss,accuracy], feed_dict=f_valid)
#         write.add_summary(val,epoch)
#         # write.add_summary(acc,epoch)
#         if epoch%25 == 0:
#             print('--------------------------------------------------')
#             print('Epoch:{} \t Validation Loss:{:.2f} , acc: {:.2f}'.format(epoch,ls,acc))
#             print('--------------------------------------------------')
#         loss_buffer.append(ls)
#         acc_buffer.append(acc)
        
#     ls,acc = sess.run([loss,accuracy],feed_dict={X:x_test,Y:y_test})
#     print('--------------------------------------------------')
#     print('final \t test Loss:{:.2f} , acc: {:.2f}'.format(ls,acc))
#     print('--------------------------------------------------')

In [15]:
train_task("./summary/test1",(x_train,y_train),(x_valid,y_valid),batch_size,
           loss,accuracy,training_op,40,train_tensorboard,merged);

--------------------------------------------------
Epoch:0 	 Validation Loss:3.03 , acc: 0.08
--------------------------------------------------
--------------------------------------------------
Epoch:25 	 Validation Loss:0.00 , acc: 1.00
--------------------------------------------------
--------------------------------------------------
final 	 test Loss:5.85 , acc: 0.62
--------------------------------------------------


In [16]:
plt.plot(list(range(len(loss_buffer))),loss_buffer)
plt.title("Loss")
plt.show()

NameError: name 'loss_buffer' is not defined

In [ ]:
with tf.Session() as sess:
    sess.run(tf.print(loss_buffer[0]))

In [ ]:
plt.plot(list(range(len(acc_buffer))),acc_buffer)
plt.title("Accuracy")
plt.show()

In [ ]:
tf.keras.utils.normalize([[1,1,1]],order=1)